In [1]:
!pip install websocket-client
!pip install numba

     |████████████████████████████████| 53 kB 1.4 MB/s 


In [2]:
import numba
import json
import timeit
import pandas as pd
import websocket

In [ ]:
import json

import pandas as pd
import websocket

# df = pd.DataFrame(columns=['foreignNotional', 'grossValue', 'homeNotional', 'price', 'side',
#                            'size', 'symbol', 'tickDirection', 'timestamp', 'trdMatchID'])

try:
    import thread
except ImportError:
    import _thread as thread
import time

global snapshot
snapshot = pd.DataFrame(columns=['price','symbol','id','side','size','timestamp_e6','cross_seq'])
global temp 
temp = {}
global temp_trade
temp_trade = {}
global delta
delta = pd.DataFrame(columns=['event_type','price','symbol','id','side','size','cross_seq','timestamp_e6'])
trade = pd.DataFrame(columns=['timestamp','trade_time_ms','symbol','trade_id','side','size','price','cross_seq','tick_direction'])
    # trade data
    # "data": [
    #     {
    #         "timestamp": "2020-01-12T16:59:59.000Z",
    #         "trade_time_ms": 1582793344685, // trade time in millisecond
    #         "symbol": "BTCUSD",
    #         "side": "Sell",
    #         "size": 328,
    #         "price": 8098,
    #         "tick_direction": "MinusTick",
    #         "trade_id": "00c706e1-ba52-5bb0-98d0-bf694bdc69f7",
    #         "cross_seq": 1052816407

def on_message(ws, message):
    global msg
    msg = json.loads(message)    
    print('message ')
    print(msg)
    with open('data.txt', 'a') as outfile:
         json.dump(msg, outfile, indent = 4)
    if msg['topic'] == 'trade.BTCUSD':
      global temp_trade
      temp_trade = pd.DataFrame(msg['data'])
      global trade      
      trade = trade.append(msg['data'],ignore_index=True)

    if msg['type'] == 'snapshot':
      global snapshot
      temp =  pd.DataFrame(msg['data'])
      temp['timestamp_e6'] = msg['timestamp_e6']
      temp['cross_seq'] = msg['cross_seq']
      snapshot = snapshot.append(temp)
    elif msg['type'] == 'delta':
      for event_type in  msg['data']:
        temp =  pd.DataFrame(msg['data'][event_type])
        temp['event_type'] = event_type
        temp['cross_seq'] = msg['cross_seq']
        temp['timestamp_e6'] = msg['timestamp_e6']
        global delta
        delta = delta.append(temp)

      temp = msg['data']
    global df
    # `ignore_index=True` has to be provided, otherwise you'll get
    # "Can only append a Series if ignore_index=True or if the Series has a name" errors
    # df = df.append(msg, ignore_index=True)


def on_error(ws, error):
    print(error)


def on_close(ws,*arg):

    print("### closed ###")


def on_open(ws):
    def run(*args):
        t_end = time.time() + 60
        send_count = 0

        while time.time() < t_end:
          # print (time.time())
          if send_count == 0: 
            ws.send('{"op": "subscribe", "args": ["orderBook_200.100ms.BTCUSD","trade.BTCUSD"]}')
            send_count =+ 1            
          elif send_count > 0:
            continue
        time.sleep(1)
        ws.close()

        print("thread terminating...")
    thread.start_new_thread(run, ())
    return


if __name__ == "__main__":
    websocket.setdefaulttimeout(5)
    ws = websocket.WebSocketApp("wss://stream-testnet.bybit.com/realtime",
                                on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
    
    
    ws.run_forever()

message 
{'topic': 'orderBook_200.100ms.BTCUSD', 'type': 'snapshot', 'data': [{'price': '47660.00', 'symbol': 'BTCUSD', 'id': 476600000, 'side': 'Buy', 'size': 40}, {'price': '47676.50', 'symbol': 'BTCUSD', 'id': 476765000, 'side': 'Buy', 'size': 100}, {'price': '47680.00', 'symbol': 'BTCUSD', 'id': 476800000, 'side': 'Buy', 'size': 40}, {'price': '47680.50', 'symbol': 'BTCUSD', 'id': 476805000, 'side': 'Buy', 'size': 8000}, {'price': '47700.00', 'symbol': 'BTCUSD', 'id': 477000000, 'side': 'Buy', 'size': 40}, {'price': '47705.00', 'symbol': 'BTCUSD', 'id': 477050000, 'side': 'Buy', 'size': 100000}, {'price': '47720.00', 'symbol': 'BTCUSD', 'id': 477200000, 'side': 'Buy', 'size': 40}, {'price': '47730.00', 'symbol': 'BTCUSD', 'id': 477300000, 'side': 'Buy', 'size': 22705}, {'price': '47740.00', 'symbol': 'BTCUSD', 'id': 477400000, 'side': 'Buy', 'size': 40}, {'price': '47744.00', 'symbol': 'BTCUSD', 'id': 477440000, 'side': 'Buy', 'size': 2500}, {'price': '47758.00', 'symbol': 'BTCUSD'

In [ ]:
trade

,timestamp,trade_time_ms,symbol,trade_id,side,size,price,cross_seq,tick_direction
0,2021-12-23T15:46:22.000Z,1640274382818,BTCUSD,216bcf4e-895f-566c-985c-923af9c63fcf,Buy,200,48760,4775819918,ZeroPlusTick
1,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,94487830-2202-5bad-a9fc-dd462480863f,Sell,6,48759.5,4775820093,MinusTick
2,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,43ae8892-477f-5e88-b795-bc0e4abef966,Sell,10,48759.5,4775820093,ZeroMinusTick
3,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,7df7b72b-5559-5bd0-a82a-84e028b0c515,Sell,20,48750,4775820093,MinusTick
4,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,28cf0587-6a95-56d0-99fe-be2c3a5c8849,Sell,20,48740,4775820093,MinusTick
5,2021-12-23T15:46:40.000Z,1640274400265,BTCUSD,1b9d3777-4d2e-56cb-b326-8223f71f86ec,Sell,244,48738,4775820093,MinusTick
6,2021-12-23T15:46:41.000Z,1640274401826,BTCUSD,1df93f97-a369-5f68-a6b8-f6ccae6772cb,Sell,2700,48738,4775820128,ZeroMinusTick
7,2021-12-23T15:46:43.000Z,1640274403520,BTCUSD,f467ca9c-8075-5880-8e77-a0d438281010,Buy,300,48739,4775820141,PlusTick
8,2021-12-23T15:46:52.000Z,1640274412581,BTCUSD,2c159acb-12cf-575a-a6a6-d7ee44f9184f,Sell,19,48740.5,4775820381,PlusTick
9,2021-12-23T15:46:54.000Z,1640274414827,BTCUSD,5c20d845-5edd-5598-ab2b-72ed3125354d,Sell,23,48744.5,4775820437,PlusTick


In [ ]:
snapshot

,price,symbol,id,side,size,timestamp_e6,cross_seq
0,47660.00,BTCUSD,476600000,Buy,40,1640274379047154,4775819907
1,47676.50,BTCUSD,476765000,Buy,100,1640274379047154,4775819907
2,47680.00,BTCUSD,476800000,Buy,40,1640274379047154,4775819907
3,47680.50,BTCUSD,476805000,Buy,8000,1640274379047154,4775819907
4,47700.00,BTCUSD,477000000,Buy,40,1640274379047154,4775819907
...,...,...,...,...,...,...,...
395,49304.00,BTCUSD,493040000,Sell,8398,1640274379047154,4775819907
396,49305.50,BTCUSD,493055000,Sell,87139,1640274379047154,4775819907
397,49311.00,BTCUSD,493110000,Sell,59084,1640274379047154,4775819907
398,49314.00,BTCUSD,493140000,Sell,7,1640274379047154,4775819907


In [ ]:
delta

,event_type,price,symbol,id,side,size,cross_seq,timestamp_e6
0,delete,47660.00,BTCUSD,476600000,Buy,NaN,4775819908,1640274379247565
0,insert,48701.00,BTCUSD,487010000,Buy,902,4775819908,1640274379247565
0,update,48763.00,BTCUSD,487630000,Sell,57820,4775819912,1640274380247486
1,update,48762.50,BTCUSD,487625000,Sell,77610,4775819912,1640274380247486
2,update,48744.00,BTCUSD,487440000,Buy,162010,4775819912,1640274380247486
...,...,...,...,...,...,...,...,...
3,update,48724.00,BTCUSD,487240000,Buy,132060,4775820817,1640274440147770
0,insert,49297.50,BTCUSD,492975000,Sell,55830,4775820817,1640274440147770
0,delete,47773.50,BTCUSD,477735000,Buy,NaN,4775820819,1640274440247826
0,update,48794.00,BTCUSD,487940000,Sell,1011,4775820819,1640274440247826


In [ ]:
snapshot.to_csv('snapshot.csv',index = False)

In [ ]:
delta.to_csv('delta.csv',index = False)

In [ ]:
trade.to_csv('trade.csv',index = False)

# Analysis

In [ ]:
snapshot

,price,symbol,id,side,size,timestamp_e6,cross_seq
0,47660.00,BTCUSD,476600000,Buy,40,1640274379047154,4775819907
1,47676.50,BTCUSD,476765000,Buy,100,1640274379047154,4775819907
2,47680.00,BTCUSD,476800000,Buy,40,1640274379047154,4775819907
3,47680.50,BTCUSD,476805000,Buy,8000,1640274379047154,4775819907
4,47700.00,BTCUSD,477000000,Buy,40,1640274379047154,4775819907
...,...,...,...,...,...,...,...
395,49304.00,BTCUSD,493040000,Sell,8398,1640274379047154,4775819907
396,49305.50,BTCUSD,493055000,Sell,87139,1640274379047154,4775819907
397,49311.00,BTCUSD,493110000,Sell,59084,1640274379047154,4775819907
398,49314.00,BTCUSD,493140000,Sell,7,1640274379047154,4775819907


In [ ]:
delta

,event_type,price,symbol,id,side,size,cross_seq,timestamp_e6
0,delete,47660.00,BTCUSD,476600000,Buy,NaN,4775819908,1640274379247565
0,insert,48701.00,BTCUSD,487010000,Buy,902,4775819908,1640274379247565
0,update,48763.00,BTCUSD,487630000,Sell,57820,4775819912,1640274380247486
1,update,48762.50,BTCUSD,487625000,Sell,77610,4775819912,1640274380247486
2,update,48744.00,BTCUSD,487440000,Buy,162010,4775819912,1640274380247486
...,...,...,...,...,...,...,...,...
3,update,48724.00,BTCUSD,487240000,Buy,132060,4775820817,1640274440147770
0,insert,49297.50,BTCUSD,492975000,Sell,55830,4775820817,1640274440147770
0,delete,47773.50,BTCUSD,477735000,Buy,NaN,4775820819,1640274440247826
0,update,48794.00,BTCUSD,487940000,Sell,1011,4775820819,1640274440247826


# Measure performance

In [ ]:
%%prun
import json

import pandas as pd
import websocket
import timeit
from timeit import default_timer as timer
import time

# df = pd.DataFrame(columns=['foreignNotional', 'grossValue', 'homeNotional', 'price', 'side',
#                            'size', 'symbol', 'tickDirection', 'timestamp', 'trdMatchID'])

try:
    import thread
except ImportError:
    import _thread as thread


    # trade data
    # "data": [
    #     {
    #         "timestamp": "2020-01-12T16:59:59.000Z",
    #         "trade_time_ms": 1582793344685, // trade time in millisecond
    #         "symbol": "BTCUSD",
    #         "side": "Sell",
    #         "size": 328,
    #         "price": 8098,
    #         "tick_direction": "MinusTick",
    #         "trade_id": "00c706e1-ba52-5bb0-98d0-bf694bdc69f7",
    #         "cross_seq": 1052816407

def update_order_book(df_snapshot, df_delta_trade_union):
  # len(df_delta_trade_union)
  # find unique list of 
  columns_list = df_snapshot.columns
  df_snapshot_updated = df_snapshot

  # temp.rename(columns={'side_delta':'side','size_x':'size_delta','side_y':'side_trade','size_y':'size_trade'})
  # temp = temp[columns_list]
  for i in range(len(df_delta_trade_union)-1):
    temp = df_delta_trade_union.loc[[i]] # 1 row only
    if temp['event_type'].iloc[-1] in ['delete','insert','update']:
      # handling the delta data
      temp = temp.rename(columns={'side_delta':'side','updated_value_delta':'size'})
      df_snapshot_updated = df_snapshot_updated.drop(df_snapshot_updated[(df_snapshot_updated['price'] == temp['price'].iloc[-1]) & (df_snapshot_updated['side'] == temp['side'].iloc[-1])].index)
      
      temp_subset = temp[columns_list]
      if temp['event_type'].iloc[-1] == 'delete':
        pass # don't append delta row if it is delete
      else:
        df_snapshot_updated = df_snapshot_updated.append(temp_subset, ignore_index=True)

    if temp['trade_id'].notna().all():
      temp = temp.rename(columns={'side':'side_delta', 'side_trade':'side','size' : 'size_delta', 'size_trade': 'size'})

      # market order cannot be found in the current snapshot, there will be 0 rows found from the filter
      # only make adjustment for the limit order here:
      if len(df_snapshot_updated[(df_snapshot_updated['price'] == temp['price'].iloc[-1]) & (df_snapshot_updated['side'] == temp['side'].iloc[-1])]) == 1 :
        temp['size'] = temp['change_value_trade'] + df_snapshot_updated[(df_snapshot_updated['price'] == temp['price'].iloc[-1]) & (df_snapshot_updated['side'] == temp['side'].iloc[-1])]['size']
        temp_subset = temp[columns_list]
        df_snapshot_updated = df_snapshot_updated.drop(df_snapshot_updated[(df_snapshot_updated['price'] == temp['price'].iloc[-1]) & (df_snapshot_updated['side'] == temp['side'].iloc[-1])].index)
        df_snapshot_updated = df_snapshot_updated.append(temp_subset, ignore_index=True)

def normalize_task (message):
  msg = json.loads(message)    
  print(msg)

  if msg['topic'] == 'trade.BTCUSD':
    temp_trade = pd.DataFrame(msg['data'])

  if msg['type'] == 'snapshot':
    temp_snapshot =  pd.DataFrame(msg['data'])
    temp_snapshot['timestamp_e6'] = msg['timestamp_e6']
    temp_snapshot['cross_seq'] = msg['cross_seq']

  elif msg['type'] == 'delta':
    for event_type in  msg['data']:
      temp_delta =  pd.DataFrame(msg['data'][event_type])
      temp_delta['event_type'] = event_type
      temp_delta['cross_seq'] = msg['cross_seq']
      temp_delta['timestamp_e6'] = msg['timestamp_e6']

  df_delta_trade_union = pd.merge(temp_delta, temp_trade,how = 'outer', on=["cross_seq", "price","symbol"])
  df_delta_trade_union = df_delta_trade_union.rename(columns={'side_x':'side_delta','size_x':'size_delta','side_y':'side_trade','size_y':'size_trade'})

  return df_delta_trade_union, df_snapshot



def on_message(ws, message):
  start = timer()
  fullstart = start
  print('start timer ')
  print(timer())  
 
  %%timeit 
  df_delta_trade_union,df_snapshot = normalize_task(message)    
  update_order_book(df_snapshot,df_delta_trade_union)

  print("end timer ")
  print(timer())  

  end = timer()
  print("Total time : %.1f ms" % (1000 * (end - fullstart)))
  global perf
  perf = perf.append(float(1000 * (end - fullstart)))


def on_error(ws, error):
    print(error)


def on_close(ws,*arg):

    print("### closed ###")


def on_open(ws):
    def run(*args):
        t_end = time.time() + 10
        send_count = 0

        while time.time() < t_end:
          # print (time.time())
          if send_count == 0: 
            ws.send('{"op": "subscribe", "args": ["orderBook_200.100ms.BTCUSD","trade.BTCUSD"]}')
            send_count =+ 1            
          elif send_count > 0:
            continue
        time.sleep(1)
        ws.close()

        print("thread terminating...")
    thread.start_new_thread(run, ())
    return


if __name__ == "__main__":
    websocket.setdefaulttimeout(5)
    ws = websocket.WebSocketApp("wss://stream-testnet.bybit.com/realtime",
                                on_open=on_open, on_message=on_message, on_error=on_error, on_close=on_close)
    
    
    ws.run_forever()

start timer 
3509.532817495
{'topic': 'orderBook_200.100ms.BTCUSD', 'type': 'snapshot', 'data': [{'price': '50317.00', 'symbol': 'BTCUSD', 'id': 503170000, 'side': 'Buy', 'size': 200}, {'price': '50324.00', 'symbol': 'BTCUSD', 'id': 503240000, 'side': 'Buy', 'size': 100}, {'price': '50329.00', 'symbol': 'BTCUSD', 'id': 503290000, 'side': 'Buy', 'size': 100}, {'price': '50331.50', 'symbol': 'BTCUSD', 'id': 503315000, 'side': 'Buy', 'size': 100}, {'price': '50333.50', 'symbol': 'BTCUSD', 'id': 503335000, 'side': 'Buy', 'size': 100}, {'price': '50339.00', 'symbol': 'BTCUSD', 'id': 503390000, 'side': 'Buy', 'size': 500}, {'price': '50339.50', 'symbol': 'BTCUSD', 'id': 503395000, 'side': 'Buy', 'size': 200}, {'price': '50343.00', 'symbol': 'BTCUSD', 'id': 503430000, 'side': 'Buy', 'size': 200}, {'price': '50345.50', 'symbol': 'BTCUSD', 'id': 503455000, 'side': 'Buy', 'size': 100}, {'price': '50350.00', 'symbol': 'BTCUSD', 'id': 503500000, 'side': 'Buy', 'size': 1000}, {'price': '50351.50', 

In [ ]:
print(timer())

1933.079590665


In [ ]:
perf

[]